#**Limpieza y Transformacion de Datos: Duplicados**


---
Tema: Limpieza y Transformacion de Datos: Duplicados

Unidad: 02 Análisis Descriptivo de los Datos

Materia: Programación para Analítica Descriptiva y Predictiva

Maestría en Inteligencia Artificial y Analítica de Datos

---

## Objetivo

Comprender la problemática de los datos duplicados, su impacto en el análisis de datos y aprender a detectarlos y tratarlos con pandas.

Las temas de esta sesión son:



*   Introducción a Datos Duplicados
* Función de Pandas `.duplicated()`
*   Detección de Datos Duplicados
* Tratamiento de Datos Duplicados
  * Eliminación de Datos Duplicados
  * Manejo Avanzado de Datos Duplicados




## **Introducción a Datos Duplicados**

Los datos duplicados son registros repetidos en un conjunto de datos. Pueden ser duplicaciones exactas/idénticas (varios registros en todas sus columnas tienen el mismo valor) o parciales (coincidencia en ciertas columnas clave). Estos pueden generarse debido a errores en la recolección, integración o procesamiento de datos.

**Ejemplo de Datos Duplicados**

\begin{array}{ccc}
\text{ID}&\text{Nombre}&\text{Edad}\\
1	&Ana&	25\\
2&	Juan&	30\\
3&	Luis&	35\\
4&	Sara&	40\\
1&	Ana	&25\\
2 &  Juan  &  30\\
2  & Juan  &  30\\
1 & Vicente & 48\\
3 & Rogelio & 40
\end{array}

En la tabla anterior, los registros idénticos (duplicidad exacta) son: [1,Ana,25] y [2, Juan, 30]. En el caso de la duplicidad parcial, dependerá de las columnas elegidas. Por columna: ID con valores 1, 2,y 3; Nombre con Ana y  Juan; y finalmente en Edad con los valores 25, 30 y 40.

---


### Problemas que generan en el análisis descriptivo y exploratorio

1. **Distorsión de métricas**: Los duplicados pueden inflar conteos, promedios y frecuencias, sesgando la interpretación de los datos.
2. **Errores en modelos predictivos**: Los algoritmos de aprendizaje automático pueden aprender patrones incorrectos al usar varias veces los mismos datos. El uso de datos similares puede generar lo que se llama un **sobreajuste** u **overfitting**.
3. **Falsas correlaciones**: Al duplicarse registros, las relaciones entre variables pueden parecer más fuertes de lo que realmente son.
4. **Aumento del costo computacional**: Un conjunto de datos con duplicados ocupa más memoria y ralentiza los procesos de análisis y modelado.

---

En el siguiente ejemplo veremos el caso de un `dataframe` con datos duplicados. Vamos a imprimir la media del salario.





In [3]:
import pandas as pd

# Crear un DataFrame con múltiples duplicados
data = {
    'ID': [1, 2, 3, 4, 1, 2, 2, 3, 5, 6, 7, 3, 5, 7, 1, 2],
    'Nombre': ['Ana', 'Juan', 'Luis', 'Sara', 'Ana', 'Juan', 'Juan', 'Luis', 'Carlos', 'Elena', 'Mario', 'Luis', 'Carlos', 'Mario', 'Ana', 'Juan'],
    'Edad': [25, 30, 35, 40, 25, 30, 30, 35, 28, 32, 29, 35, 28, 29, 25, 30],
    'Salario': [50000, 60000, 55000, 70000, 50000, 60000, 60000, 55000, 48000, 51000, 53000, 55000, 48000, 53000, 50000, 60000]
}

# Convertir en DataFrame
df= pd.DataFrame(data)

# Mostrar el DataFrame
print(df)

print('\nLa media del salario es con duplicados es ',df['Salario'].mean())

    ID  Nombre  Edad  Salario
0    1     Ana    25    50000
1    2    Juan    30    60000
2    3    Luis    35    55000
3    4    Sara    40    70000
4    1     Ana    25    50000
5    2    Juan    30    60000
6    2    Juan    30    60000
7    3    Luis    35    55000
8    5  Carlos    28    48000
9    6   Elena    32    51000
10   7   Mario    29    53000
11   3    Luis    35    55000
12   5  Carlos    28    48000
13   7   Mario    29    53000
14   1     Ana    25    50000
15   2    Juan    30    60000

La media del salario es con duplicados es  54875.0


Si miramos los duplicados de la columna ID, encontramos que los valores 1, 2, 3 y 5 presentan valores repetidos. Vamos a ver cómo afecta a la media del salario si quitamos los repetidos y solo dejamos una coincidencia de cada uno de registros. Observa lo que pasa cuando quitamos los duplicados. La media del salario no coincide.

In [4]:
data2 = {
    'ID': [1, 2, 3, 4, 5, 6, 7],
    'Nombre': ['Ana', 'Juan', 'Luis', 'Sara', 'Carlos', 'Elena', 'Mario'],
    'Edad': [25, 30, 35, 40, 28, 32, 29],
    'Salario': [50000, 60000, 55000, 70000, 48000, 51000, 53000]
}

# Convertir en DataFrame
dfND = pd.DataFrame(data2)

# Mostrar el DataFrame
print(dfND)

print('\nLa media del salario es sin duplicados es ',dfND['Salario'].mean())

   ID  Nombre  Edad  Salario
0   1     Ana    25    50000
1   2    Juan    30    60000
2   3    Luis    35    55000
3   4    Sara    40    70000
4   5  Carlos    28    48000
5   6   Elena    32    51000
6   7   Mario    29    53000

La media del salario es sin duplicados es  55285.71428571428


Es importante mencionar que en esta primera parte de limpieza de datos, el concepto de datos idénticos que adoptaremos será en términos de sus valores. Esto debido a que en modelos predictivos este concepto puede variar un poco, en donde, la similitud entre dos datos puede estar determinada por una métrica de distancia o por si el modelo es capaz de reconocer su etiqueta. Por ejemplo, en aprendizaje activo (active learning), si el clasificador puede predecir la etiqueta de un ejemplo, entonces este no formará parte del proceso de aprendizaje.


## Función en Pandas para Identificar y Manejar Duplicados `.duplicated()`


`.duplicated()` es una función poderosa en *Pandas* que permite identificar y manejar duplicados de manera flexible. Se puede aplicar a todo el `Dataframe` (duplicidad exacta) o a columnas específicas (duplicidad parcial). Esta función retorna una serie booleana donde:

* `False` indica que la fila no está duplicada (o es la primera aparición).
* `True` indica que la fila es un duplicado (es decir, ya apareció antes en el `DataFrame`).

La sintaxis básica es:

```python
df.duplicated(subset=None, keep='first')
```
donde,

* `subset` es opcional y recibe una lista con los nombres de las columnas en las que se buscarán los duplicados
* `keep` es opcional y define qué ocurrencias de los duplicados se marcan como `True`.

## Duplicidad Exacta: `.duplicated()` con `keep`

Vamos a usar diferentes valores que puede tomar el párametro `keep` para distinguir qué duplicados se marcan como `True`:
:
  * `'first'` (por defecto): Marca como `True` todas las filas duplicadas *excepto la primera coincidencia*
  * `'last'`: Marca todas las filas duplicadas como `True` *excepto la última coincidencia*
  * `False`: Marca *todas* las filas duplicadas, incluida la primera

Veamos un ejemplo sencillo donde la detección se hace de forma exacta, es decir, en todo el `Dataframe`.


In [49]:
import pandas as pd


print('Imprime el Dataframe original')
print(df)
# Identificar todas filas duplicadas
#Observa que no imprime True en la en la Fila 0, ya que por defecto es first'
print('\n Marca como True las filas en donde la coincidencia es total excepto la primera coincidencia')
print(df.duplicated())

#
print('\n Similar al anterior. Marca como True todas las filas duplicadas excepto la primera coincidencia')
print(df.duplicated(keep='first'))


print('\n Marca como True todas las filas duplicadas excepto la última coincidencia')
print(df.duplicated(keep='last'))

print('\n Marca como True todas las filas duplicadas incluida la primera')
print(df.duplicated(keep=False))



Imprime el Dataframe original
    ID  Nombre  Edad  Salario
0    1     Ana    25    50000
1    2    Juan    30    60000
2    3    Luis    35    55000
3    4    Sara    40    70000
4    1     Ana    25    50000
5    2    Juan    30    60000
6    2    Juan    30    60000
7    3    Luis    35    55000
8    5  Carlos    28    48000
9    6   Elena    32    51000
10   7   Mario    29    53000
11   3    Luis    35    55000
12   5  Carlos    28    48000
13   7   Mario    29    53000
14   1     Ana    25    50000
15   2    Juan    30    60000

 Marca como True las filas en donde la coincidencia es total excepto la primera coincidencia
0     False
1     False
2     False
3     False
4      True
5      True
6      True
7      True
8     False
9     False
10    False
11     True
12     True
13     True
14     True
15     True
dtype: bool

 Similar al anterior. Marca como True todas las filas duplicadas excepto la primera coincidencia
0     False
1     False
2     False
3     False
4      True
5   

## Duplicidad Parcial: `.duplicated()` con `subset`

El parámetro `subset` en  permite especificar qué columnas se deben considerar al detectar duplicados. Esto es útil cuando queremos identificar valores exactos en algunas columnas específicas en lugar de comparar todas las columnas del `DataFrame`.

La sintaxis básica es:

```python
df.duplicated(subset=['columna1', 'columna2'])
```
Adicionalmente se puede usar en combinación con `keep`.



In [50]:
# Detectar duplicados en la columna 'ID'
print('\n Detectar duplicados en la columna ID')
print(df.duplicated(subset=['ID'], keep='first'))

# Detectar duplicados en las columnas 'Nombre'
print('\n Detectar duplicados en la columna Nombre. No se marca la última aparición')
print(df.duplicated(subset=['Nombre'], keep='last'))

# Detectar duplicados en las columnas 'Nombre y Edad'
print('\n Detectar duplicados en las columnas Nombre y Edad. Marca todas las filas incluida la primera')
print(df.duplicated(subset=['Nombre', 'Edad'], keep=False))



 Detectar duplicados en la columna ID
0     False
1     False
2     False
3     False
4      True
5      True
6      True
7      True
8     False
9     False
10    False
11     True
12     True
13     True
14     True
15     True
dtype: bool

 Detectar duplicados en la columna Nombre. No se marca la última aparición
0      True
1      True
2      True
3     False
4      True
5      True
6      True
7      True
8      True
9     False
10     True
11    False
12    False
13    False
14    False
15    False
dtype: bool

 Detectar duplicados en las columnas Nombre y Edad. Marca todas las filas incluida la primera
0      True
1      True
2      True
3     False
4      True
5      True
6      True
7      True
8      True
9     False
10     True
11     True
12     True
13     True
14     True
15     True
dtype: bool


## Contar el número total de registros duplicados y las veces que se repite cada valor

La forma más sencilla es usando `.sum()`, la cual, cuando se agrega a `.duplicated()` (devuelve una serie booleana) cuenta cuántos valores `True` hay en la serie, lo que nos da el número total de registros duplicados.

In [5]:
# Contar filas duplicadas en todo el DataFrame
num_duplicados = df.duplicated().sum()
print(f"Número total de registros duplicados sin tener en cuenta la primera aparación: {num_duplicados}")

num_duplicados = df.duplicated(keep='last').sum()
print(f"Número total de registros duplicados sin tener en cuenta la última aparación: {num_duplicados}")

num_duplicados = df.duplicated(keep=False).sum()
print(f"Número total de registros duplicados tiene en cuenta todas las apariciones incluyendo la primera: {num_duplicados}")

Número total de registros duplicados sin tener en cuenta la primera aparación: 9
Número total de registros duplicados sin tener en cuenta la última aparación: 9
Número total de registros duplicados tiene en cuenta todas las apariciones incluyendo la primera: 14


Podemos contar duplicados considerando solo una columna o múltiples.

In [52]:
num_duplicados_id = df.duplicated(subset=['ID'], keep=False).sum()
print(f"Número Total de registros con ID duplicado (todas las apariciones): {num_duplicados_id}")

num_duplicados_multi = df.duplicated(subset=['Nombre', 'Edad'], keep=False).sum()
print(f"Número Total de registros duplicados considerando Nombre y Edad (todas las apariciones y se repiten juntos): {num_duplicados_multi}")


Número Total de registros con ID duplicado (todas las apariciones): 14
Número Total de registros duplicados considerando Nombre y Edad (todas las apariciones y se repiten juntos): 14


Si deseamos contar los valores que se repiten más de una vez, usamos la función vista en otra clase `value_counts()`.

In [53]:
conteo_duplicados = df.value_counts()
print("Todos los valores únicos y sus respectivas cantidades en todas las Columnas")
print(conteo_duplicados)
conteo_duplicados = df['Nombre'].value_counts()
print("Todos los valores únicos y sus respectivas cantidades en la columna Nombre")
print(conteo_duplicados)
print("Valores únicos que se repiten más de una vez")
print(conteo_duplicados[conteo_duplicados > 1])



Todos los valores únicos y sus respectivas cantidades en todas las Columnas
ID  Nombre  Edad  Salario
2   Juan    30    60000      4
1   Ana     25    50000      3
3   Luis    35    55000      3
5   Carlos  28    48000      2
7   Mario   29    53000      2
4   Sara    40    70000      1
6   Elena   32    51000      1
Name: count, dtype: int64
Todos los valores únicos y sus respectivas cantidades en la columna Nombre
Nombre
Juan      4
Ana       3
Luis      3
Carlos    2
Mario     2
Sara      1
Elena     1
Name: count, dtype: int64
Valores únicos que se repiten más de una vez
Nombre
Juan      4
Ana       3
Luis      3
Carlos    2
Mario     2
Name: count, dtype: int64


Si queremos obtener proporciones en lugar de conteos, podemos usar `normalize=True`

In [54]:
conteo_duplicados = df['ID'].value_counts(normalize=True)

print("Proporción de valores duplicados")
print(conteo_duplicados)

Proporción de valores duplicados
ID
2    0.2500
1    0.1875
3    0.1875
5    0.1250
7    0.1250
4    0.0625
6    0.0625
Name: proportion, dtype: float64


## Filtrar o Eliminar Datos Duplicados

### Filtrar
Podemos obtener las filas en las que todas sus valores en cada columna es igual o usar `subset` para obtener solo las filas que tienen valores duplicados en una columna específica.


In [55]:
print("Filtrar: Todos los registros con datos similares en todas las columnas")
print(df[df.duplicated( keep=False)])

print("Filtrar: Todos los registros con datos similares en la columna Edad")
print(df[df.duplicated(subset=['Edad'], keep=False)])

print("Filtrar: Todos los registros (todas las columnas) sin duplicados")
print(df[~df.duplicated(keep=False)])


Filtrar: Todos los registros con datos similares en todas las columnas
    ID  Nombre  Edad  Salario
0    1     Ana    25    50000
1    2    Juan    30    60000
2    3    Luis    35    55000
4    1     Ana    25    50000
5    2    Juan    30    60000
6    2    Juan    30    60000
7    3    Luis    35    55000
8    5  Carlos    28    48000
10   7   Mario    29    53000
11   3    Luis    35    55000
12   5  Carlos    28    48000
13   7   Mario    29    53000
14   1     Ana    25    50000
15   2    Juan    30    60000
Filtrar: Todos los registros con datos similares en la columna Edad
    ID  Nombre  Edad  Salario
0    1     Ana    25    50000
1    2    Juan    30    60000
2    3    Luis    35    55000
4    1     Ana    25    50000
5    2    Juan    30    60000
6    2    Juan    30    60000
7    3    Luis    35    55000
8    5  Carlos    28    48000
10   7   Mario    29    53000
11   3    Luis    35    55000
12   5  Carlos    28    48000
13   7   Mario    29    53000
14   1     Ana    25 

Para eliminar registros duplicados usamos la función `drop_duplicates()`. La sintaxis es la siguiente


```python
df.drop_duplicates(subset=None, keep='first', inplace=False)
```
Donde los parámetros pueden tomar los siguientes valores:

* `subset=None`: Considera todas las columnas para detectar duplicados.
  * Si se especifica `subset=['col1', 'col2']`, solo se eliminan duplicados basados en esas columnas.
* `keep='first' (por defecto): Mantiene la primera aparición y elimina las demás.
* `keep='last'`: Mantiene la última aparición y elimina las demás.
* `keep=False`: Elimina todas las filas duplicadas.
* `inplace=False`: Si es `True`, modifica el `DataFrame` original sin necesidad de reasignarlo.



In [56]:
# Eliminar duplicados y conservar solo la primera aparición
df_sin_duplicados = df.drop_duplicates()
print('\nEliminar duplicados y conservar solo la primera aparición')
print(df_sin_duplicados)

#eliminar duplicados considerando la columna "ID"
print('\nEliminar duplicados considerando la columna ID y conserva la primera aparición')
df_sin_duplicados = df.drop_duplicates(subset=['ID'])
print(df_sin_duplicados)


#conservar la última ocurrencia de cada duplicado
print('\nEliminar duplicados conservando la última ocurrencia de cada duplicado')
df_sin_duplicados = df.drop_duplicates(keep='last')
print(df_sin_duplicados)

#eliminar todas las filas que tienen duplicados
print('\nEliminar todas las filas que tienen duplicados')
df_sin_duplicados = df.drop_duplicates(keep=False)
print(df_sin_duplicados)


Eliminar duplicados y conservar solo la primera aparición
    ID  Nombre  Edad  Salario
0    1     Ana    25    50000
1    2    Juan    30    60000
2    3    Luis    35    55000
3    4    Sara    40    70000
8    5  Carlos    28    48000
9    6   Elena    32    51000
10   7   Mario    29    53000

Eliminar duplicados considerando la columna ID y conserva la primera aparición
    ID  Nombre  Edad  Salario
0    1     Ana    25    50000
1    2    Juan    30    60000
2    3    Luis    35    55000
3    4    Sara    40    70000
8    5  Carlos    28    48000
9    6   Elena    32    51000
10   7   Mario    29    53000

Eliminar duplicados conservando la última ocurrencia de cada duplicado
    ID  Nombre  Edad  Salario
3    4    Sara    40    70000
9    6   Elena    32    51000
11   3    Luis    35    55000
12   5  Carlos    28    48000
13   7   Mario    29    53000
14   1     Ana    25    50000
15   2    Juan    30    60000

Eliminar todas las filas que tienen duplicados
   ID Nombre  Edad  S